In [1]:
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/wbtc-ctc/")

WBTC = []

def scrape_page():
    # Locate elements  (container)
    routes = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routes:
        try:
            # Extract route name and link from the route element
            route_name_element = route.find_element(By.XPATH, ".//div[@class='route_details']")
            route_link_element = route.find_element(By.XPATH, ".//a")

            route_name = route_name_element.text
            route_link = route_link_element.get_attribute('href')

            # Append extracted data to list
            WBTC.append({
                'route_name': route_name,
                'route_link': route_link
            })

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Print the scraped data
for entry in WBTC:
    print(entry)

# Convert to DataFrame and save to CSV
df = pd.DataFrame(WBTC)
df.to_csv("wbtc_details.csv", index=False)

# Close the driver
driver.quit()

df.head()  # Display the first few rows of the DataFrame


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
{'route_name': 'Durgapur (West Bengal) to Kolkata\nFrom INR 135', 'route_link': 'https://www.redbus.in/bus-tickets/durgapur-to-kolkata'}
{'route_name': 'Digha to Barasat (West Bengal)\nFrom INR 161', 'route_link': 'https://www.redbus.in/bus-tickets/digha-to-barasat-west-bengal'}
{'route_name': 'Barasat (West Bengal) to Digha\nFrom INR 161', 'route_link': 'https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha'}
{'route_name': 'Suri to Kolkata\nFrom INR 435', 'route_link': 'https://www.redbus.in/bus-tickets/suri-to-kolkata'}
{'route_name': 'Kolkata to Digha\nFrom INR 145', 'route_link': 'https://www.redbus.in/bus-tickets/kolkata-to-digha'}
{'route_name': 'Bolpur (West Bengal) to Kolkata\nFrom INR 375', 'route_link': 'https://www.redbus.in/bus-tickets/bolpur-west-bengal-to-ko

,route_name,route_link
0,Durgapur (West Bengal) to Kolkata\nFrom INR 135,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Digha to Barasat (West Bengal)\nFrom INR 161,https://www.redbus.in/bus-tickets/digha-to-bar...
2,Barasat (West Bengal) to Digha\nFrom INR 161,https://www.redbus.in/bus-tickets/barasat-west...
3,Suri to Kolkata\nFrom INR 435,https://www.redbus.in/bus-tickets/suri-to-kolkata
4,Kolkata to Digha\nFrom INR 145,https://www.redbus.in/bus-tickets/kolkata-to-d...


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# Initialize the webdriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)

# Define the CSV file path and read URLs and route names
csv_file_path = r"C:\Users\sonur\OneDrive\Documents\vs code\Details\wbtc_details.csv"
try:
    url_df = pd.read_csv(csv_file_path, on_bad_lines='skip')  # Skip bad lines to avoid errors
    urls = url_df[['route_name', 'route_link']].to_dict(orient='records')
    print(f"URLs found: {len(urls)}")
except Exception as e:
    print(f"Error reading CSV file: {e}")
    urls = []

# Define a list to store all bus details data
bus_details = []

# Function to scrape bus details from the URL
def scrape_bus_details(route_name, url):
    print(f"Scraping data from: {url}")
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

    try:
        bus_containers = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".clearfix.bus-item-details")))
        for container in bus_containers:
            try:
                # Extract bus details
                bus_name = container.find_element(By.CSS_SELECTOR, '.travels.lh-24.f-bold.d-color').text.strip() or 'N/A'
                bus_type = container.find_element(By.CSS_SELECTOR, '.bus-type.f-12.m-top-16.l-color.evBus').text.strip() or 'N/A'
                departing_time = container.find_element(By.CSS_SELECTOR, '.dp-time.f-19.d-color.f-bold').text.strip() or 'N/A'
                duration = container.find_element(By.CSS_SELECTOR, '.dur.l-color.lh-24').text.strip() or 'N/A'
                reaching_time = container.find_element(By.CSS_SELECTOR, '.bp-time.f-19.d-color.disp-Inline').text.strip() or 'N/A'
                price = container.find_element(By.CSS_SELECTOR, '.fare.d-block span.f-19.f-bold').text.strip() or 'N/A'
                
                # Attempt to find the star rating element
                try:
                    star_rating = container.find_element(By.CSS_SELECTOR, '.rating-sec .rating span').text.strip() or 'N/A'
                except NoSuchElementException:
                    star_rating = 'N/A'
                
                seat_availability = container.find_element(By.CSS_SELECTOR, '.seat-left').get_attribute('innerText').strip() or 'N/A'
               
                bus_details.append({
                    "route_name": route_name,
                    "route_link": url,
                    "bus_name": bus_name,
                    "bus_type": bus_type,
                    "departure_time": departing_time,
                    "duration": duration,
                    "arrival_time": reaching_time,
                    "price": price,
                    "star_rating": star_rating,
                    "seat_availability": seat_availability
                })
            except NoSuchElementException as e:
                print(f"Element not found: {e}")
            except Exception as e:
                print(f"An unexpected error occurred while processing a bus container: {e}")
    except TimeoutException:
        print(f"Timed out waiting for bus containers on page: {url}")
    except Exception as e:
        print(f"An error occurred while scraping {url}: {e}")

# Iterate through each URL and scrape data
for item in urls:
    scrape_bus_details(item['route_name'], item['route_link'])

# Convert to DataFrame and save to CSV
df = pd.DataFrame(bus_details)
df.to_csv("wbtc_f.csv", index=False)

# Print the scraped bus details
print(df.head() if not df.empty else "No data to display")

# Close the driver
driver.quit()


URLs found: 41
Scraping data from: https://www.redbus.in/bus-tickets/durgapur-to-kolkata
Scraping data from: https://www.redbus.in/bus-tickets/digha-to-barasat-west-bengal
Scraping data from: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha
Scraping data from: https://www.redbus.in/bus-tickets/suri-to-kolkata
Timed out waiting for bus containers on page: https://www.redbus.in/bus-tickets/suri-to-kolkata
Scraping data from: https://www.redbus.in/bus-tickets/kolkata-to-digha
Scraping data from: https://www.redbus.in/bus-tickets/bolpur-west-bengal-to-kolkata
Scraping data from: https://www.redbus.in/bus-tickets/kolkata-to-suri
Timed out waiting for bus containers on page: https://www.redbus.in/bus-tickets/kolkata-to-suri
Scraping data from: https://www.redbus.in/bus-tickets/digha-to-kolkata
Scraping data from: https://www.redbus.in/bus-tickets/kolkata-to-durgapur
Scraping data from: https://www.redbus.in/bus-tickets/habra-to-digha
Scraping data from: https://www.redbus.in/bu

In [2]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Path to the CSV file
csv_file_path = r"C:\Users\sonur\OneDrive\Documents\vs code\10 states\Done\wbtc_f.csv"
# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Print the column names to verify
print("Columns:", df.columns)

# Handle missing values and ensure all empty fields are filled
df['seats_availability'] = df['seats_availability'].fillna('N/A')
df['route_name'] = df['route_name'].fillna('N/A')
df['route_link'] = df['route_link'].fillna('N/A')
df['bus_name'] = df['bus_name'].fillna('N/A')
df['bus_type'] = df['bus_type'].fillna('N/A')
df['departure_time'] = df['departure_time'].fillna('00:00:00')
df['arrival_time'] = df['arrival_time'].fillna('00:00:00')
df['duration'] = df['duration'].fillna('N/A')
df['price'] = df['price'].fillna(0.0)
df['star_rating'] = df['star_rating'].fillna(0.0)

# Ensure time columns are strings and fill any conversion issues with '00:00:00'
df['departure_time'] = pd.to_datetime(df['departure_time'], format='%H:%M:%S', errors='coerce').fillna(pd.Timestamp('00:00:00')).dt.strftime('%H:%M:%S')
df['arrival_time'] = pd.to_datetime(df['arrival_time'], format='%H:%M:%S', errors='coerce').fillna(pd.Timestamp('00:00:00')).dt.strftime('%H:%M:%S')

# Print DataFrame to verify all fields are filled
print("DataFrame after filling missing values:")
print(df.head(10))

# MySQL database connection
try:
    connection = mysql.connector.connect(
        host='localhost',
        database='red_bus',
        user='root',
        password='Vinayak@12'
    )

    if connection.is_connected():
        cursor = connection.cursor()

        # Create table if it doesn't exist with the correct schema
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS wbtc_10 (
            id INT AUTO_INCREMENT PRIMARY KEY,
            route_name TEXT,
            route_link TEXT,
            bus_name TEXT,
            bus_type TEXT,
            departure_time TIME,
            duration TEXT,
            arrival_time TIME,
            price DECIMAL(10, 2),
            star_rating FLOAT,
            seats_availability VARCHAR(255)
        )
        '''
        cursor.execute(create_table_query)
        connection.commit()

        # Insert data into the table
        for index, row in df.iterrows():
            try:
                insert_query = '''
                INSERT INTO wbtc_10 (route_name, route_link, bus_name, bus_type, departure_time, duration, arrival_time, price, star_rating, seats_availability)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                '''
                # Print row data for debugging
                print("Inserting row:", row)
                cursor.execute(insert_query, (
                    row['route_name'],
                    row['route_link'],
                    row['bus_name'],
                    row['bus_type'],
                    row['departure_time'],
                    row['duration'],
                    row['arrival_time'],
                    row['price'],
                    row['star_rating'],
                    row['seats_availability']
                ))

            except Error as e:
                print(f"Error inserting row {index}: {e}")

        connection.commit()
        print("Data inserted successfully")

except Error as e:
    print(f"Error: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed")


Columns: Index(['route_name', 'route_link', 'bus_name', 'bus_type', 'departure_time',
       'duration', 'arrival_time', 'price', 'star_rating',
       'seats_availability'],
      dtype='object')
DataFrame after filling missing values:
                                          route_name  \
0  Durgapur (West Bengal) to Kolkata\r\nFrom INR 135   
1  Durgapur (West Bengal) to Kolkata\r\nFrom INR 135   
2  Durgapur (West Bengal) to Kolkata\r\nFrom INR 135   
3  Durgapur (West Bengal) to Kolkata\r\nFrom INR 135   
4  Durgapur (West Bengal) to Kolkata\r\nFrom INR 135   
5     Digha to Barasat (West Bengal)\r\nFrom INR 161   
6     Digha to Barasat (West Bengal)\r\nFrom INR 161   
7     Digha to Barasat (West Bengal)\r\nFrom INR 161   
8     Digha to Barasat (West Bengal)\r\nFrom INR 161   
9     Digha to Barasat (West Bengal)\r\nFrom INR 161   

                                          route_link  \
0  https://www.redbus.in/bus-tickets/durgapur-to-...   
1  https://www.redbus.in/bus-ticke